## **Task 1: Feature Engineering**



##### **Load & Prepare the Dataset**

In [ ]:
import numpy as np
import pandas as pd
from joblib import dump

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
os.listdir('/content')

In [ ]:
# Load training data
df = pd.read_csv(
    "train_FD001.txt",
    sep=r"\s+",
    header=None
) # Read the file and split columns wherever there are one or more spaces or tabs, and assume there is no header row.

In [ ]:
# Column names based on NASA documentation
cols = (
    ["engine_id", "cycle"] +
    [f"op_setting_{i}" for i in range(1, 4)] +
    [f"sensor_{i}" for i in range(1, 22)]
)

In [ ]:
df.columns = cols

In [ ]:
df.head()

##### **Create Remaining Useful Life (RUL)**

In [ ]:
# Max cycle per engine
max_cycle = df.groupby("engine_id")["cycle"].max()

In [ ]:
# Map max cycle
df["max_cycle"] = df["engine_id"].map(max_cycle)

In [ ]:
# Remaining Useful Life
df["RUL"] = df["max_cycle"] - df["cycle"]

##### **Create 24-Hour Failure Label (Classification Target)**

In [ ]:
FAILURE_WINDOW = 24  # 24 hours / cycles

In [ ]:
df["failure_next_24hrs"] = (df["RUL"] <= FAILURE_WINDOW).astype(int) # If Remaining Useful Life (RUL) ≤ 24 cycles, failure will happen within the next 24 hours

In [ ]:
df[['RUL','failure_next_24hrs']]

#### **Feature Engineering**

##### **A. Rolling Mean & Standard Deviation(Last 1, 6, 12 hours)**

In [ ]:
WINDOWS = [6, 12] # rolling time windows
SENSORS = [f"sensor_{i}" for i in range(1, 22)] # list of sensor columns

Why 1 is NOT included in WINDOWS = [6, 12]?
* The rolling mean of 1 value = the value itself
* So this feature is identical to the original sensor
* It adds no new pattern or trend
* Standard deviation needs at least 2 values
* With 1 value → result is NaN

In [ ]:
feature_dict = {} # Collect features in a dictionary which helps to generate features WITHOUT inserting into df

In [ ]:
for sensor in SENSORS:
    for w in WINDOWS:
        feature_dict[f"{sensor}_roll_mean_{w}"] = (
            df.groupby("engine_id")[sensor] # Each engine has its own independent life cycle. Rolling stats are calculated only within the same engine
              .rolling(window=w) # Look at the previous w time steps, including the current one.
              .mean() # it captures: Overall trend & Smooths noisy sensor data
              .reset_index(level=0, drop=True)# groupby().rolling() creates a MultiIndex. Pandas can’t assign it directly to the DataFrame .Removes the engine_id index level & Aligns values correctly with original rows
        )

        feature_dict[f"{sensor}_roll_std_{w}"] = (
            df.groupby("engine_id")[sensor]
              .rolling(window=w)
              .std() # it captures: Variability / instability & Sudden fluctuations often indicate degradation
              .reset_index(level=0, drop=True)
        )
# For every sensor and every time window, it creates rolling statistical features (mean and standard deviation) separately for each engine.
# This helps the ML model understand trends and variability in sensor behavior over time.

In [ ]:
rolling_features = pd.DataFrame(feature_dict)

In [ ]:
df = pd.concat([df, rolling_features], axis=1)

### **B. Exponential Moving Average (EMA)**

EMA gives more weight to recent sensor values.

In [ ]:
ema_features = {} # It helps to stores the generated EMA feature in a dictionary instead of directly adding it to df

In [ ]:
for sensor in SENSORS: # Loops through all sensor columns
    ema_features[f"{sensor}_ema_6"] = (
        df.groupby("engine_id")[sensor] # Groups data by engine. Ensures EMA is calculated per engine lifecycle
          .ewm(span=6, adjust=False) # Applies Exponential Weighted Moving Average. span=6 → recent 6 time steps get higher weight. adjust=False → uses recursive EMA formula (standard in ML & signal processing)
          .mean() # Computes the EMA values. Despite the name, EMA is not a simple average—it weights recent values more heavily.
          .reset_index(level=0, drop=True) # Removes the engine_id index created by groupby()
    )

    ema_features[f"{sensor}_ema_12"] = (
        df.groupby("engine_id")[sensor]
          .ewm(span=12, adjust=False)
          .mean()
          .reset_index(level=0, drop=True)
    )

This code creates Exponential Moving Average (EMA) features for each sensor, calculated separately for each engine, using two time windows:

* EMA(6) → short-term behavior

* EMA(12) → medium-term behavior

These features help the model detect early degradation patterns in sensor readings.

In [ ]:
ema_df = pd.DataFrame(ema_features)

In [ ]:
df = pd.concat([df, ema_df], axis=1)

### **C. Lag Features (t-1, t-2)**

Lag features capture temporal dependency.

This creates lag (historical) features for each sensor so the ML model can learn:

How past sensor values influence future failures

Lag features are essential in time-series prediction and prevent data leakage.

In [ ]:
LAGS = [1, 2] #Lag-1 → previous time step
# Lag-2 → two time steps ago

In [ ]:
lag_features = {}

In [ ]:
for sensor in SENSORS:
    for lag in LAGS:
        lag_features[f"{sensor}_lag_{lag}"] = (
            df.groupby("engine_id")[sensor]
              .shift(lag) # Shifts sensor values backward in time & Creates historical context
        )

In [ ]:
lag_df = pd.DataFrame(lag_features)

In [ ]:
df = pd.concat([df, lag_df], axis=1)

In [ ]:
df.columns

### **Handle Missing Values**


Rolling and lag features create NaN values at the beginning of each time series.

The maximum window or lag tells us how many initial rows must be dropped to remove all NaNs safely.

In [ ]:
MAX_WINDOW = max(max(WINDOWS), max(LAGS)) #finds the largest value among all rolling window sizes (WINDOWS) and lag steps (LAGS)

In [ ]:
MAX_WINDOW

In [ ]:
df = df[df.groupby("engine_id").cumcount() >= MAX_WINDOW].reset_index(drop=True)
#removes the first few rows of each engine where time-series features (rolling, lag, EMA) are not fully available.
#cumcount():It counts rows within each group, starting from 0, and increases by 1 for every new row in that group.

### **Select Final Feature Set**

In [ ]:
FEATURE_COLUMNS = [
    col for col in df.columns
    if "sensor_" in col and
    ("roll" in col or "ema" in col or "lag" in col)
]

In [ ]:
X = df[FEATURE_COLUMNS]
y = df["failure_next_24hrs"]

In [ ]:
X.isna().sum().sum()

### **Efficient Serialization Using joblib**


In [ ]:
dump(X, "X_features_FD001.joblib")# dump() serializes (converts) Python objects into binary files. Files are saved on disk with .joblib extension
dump(y, "y_labels_FD001.joblib")

In [ ]:
from joblib import load

In [ ]:
X = load("X_features_FD001.joblib") # Reads binary .joblib files. Reconstructs the original Python objects in memory
y = load("y_labels_FD001.joblib")

## **Task 2: Modeling**


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
#stratify=y: Ensures failures appear in both train & test. Mandatory for <1% imbalance

In [ ]:
X_train.head()

### **Baseline Model: Logistic Regression**


In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()), #Scales all features to the same range so Logistic Regression works properly

    ("smote", SMOTE(
        sampling_strategy="auto", #Generates synthetic samples for the minority class to balance the dataset.(applied only on training data, no data leakage).
        random_state=42
    )),

    ("model", LogisticRegression(
        max_iter=1000, #ensures convergence
        class_weight=None, #not needed because SMOTE balances classes
        n_jobs=-1 #uses all CPU cores for faster training
    ))
])

In [ ]:
logreg_pipeline.fit(X_train, y_train)

In [ ]:
#Evaluate Logistic Regression (PR-AUC)
from sklearn.metrics import average_precision_score, precision_recall_curve

y_probs_lr = logreg_pipeline.predict_proba(X_test)[:, 1] #Selects only the probability of failure (class = 1)
pr_auc_lr = average_precision_score(y_test, y_probs_lr)
#calculates the probability of positive class predictions and evaluates the model using precision-recall performance.

print("Logistic Regression PR-AUC:", pr_auc_lr)

### **Production Model: XGBoost**

###### **Hyperparameter Tuning via GridSearchCV**

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# Count class distribution
negative_samples = (y_train == 0).sum()
positive_samples = (y_train == 1).sum() #Counts how many non-failure (0) and failure (1) samples are in the training data.

scale_pos_weight = negative_samples / positive_samples #Computes how much more weight the model should give to the rare failure class

print("scale_pos_weight:", scale_pos_weight)

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, average_precision_score

In [ ]:
# Define Pipeline
xgb_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("xgb", XGBClassifier(
        objective="binary:logistic",
        eval_metric="aucpr",
        scale_pos_weight=scale_pos_weight,
        n_jobs=-1,
        random_state=42
    ))
])

In [ ]:
# Hyperparameter Grid
param_grid = {
    "xgb__n_estimators": [200, 400],
    "xgb__max_depth": [4, 6],
    "xgb__learning_rate": [0.05, 0.1],
    "xgb__subsample": [0.8],
    "xgb__colsample_bytree": [0.8],
    "xgb__min_child_weight": [1, 5],
    "xgb__gamma": [0, 1]
}

In [ ]:
# GridSearchCV Setup
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) #Splits data into 3 folds.Stratified → keeps class imbalance ratio the same in each fold. shuffle=True → randomizes samples

grid_search = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid, #Tries all parameter combinations in param_grid
    scoring="average_precision",
    cv=cv, #Uses stratified cross-validation
    n_jobs=-1,
    verbose=2,
    error_score="raise"  # helps debugging
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
#Best Parameters
print("Best Parameters:")
print(grid_search.best_params_)

In [ ]:
#Final Model (GridSearch Best)
best_xgb_grid = grid_search.best_estimator_

In [ ]:
#Evaluate on Test Data
y_pred_proba = best_xgb_grid.predict_proba(X_test)[:, 1]

pr_auc = average_precision_score(y_test, y_pred_proba)
print("Test PR-AUC:", pr_auc)

Precision–Recall Curve

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(6, 4))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid(True)
plt.show()

The Precision–Recall curve shows excellent model performance with a PR-AUC of 0.9943. Precision remains close to 1.0 across most recall levels, indicating that the model accurately detects failures while minimizing false alarms.

## **Task 3: Explainability Using SHAP**

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
#Extract Trained XGBoost Model
xgb_model = best_xgb_grid.named_steps["xgb"] #SHAP needs the actual trained model, not the full pipeline.

In [ ]:
#Create SHAP Explainer
explainer = shap.TreeExplainer(xgb_model)

In [ ]:
#Compute SHAP Values
X_sample = X_test.sample(200, random_state=42)

shap_values = explainer.shap_values(X_sample)

**Local Explainability (Single Engine Prediction)**

In [ ]:
engine_index = 10  # choose any index
shap.force_plot( #Creates a force plot
    explainer.expected_value, #This is the average prediction of the model
    shap_values[engine_index], #SHAP values tell how much each feature contributed
    X_sample.iloc[engine_index], #Provides the actual sensor readings/features for engine 10
    matplotlib=True
)

**Base Value**

* The base value (shown near 0) is the average prediction of the model across all engines.

* If no feature information was given, the model would predict this value.

**Final Prediction (f(x) = 7.80)**

* After considering all features, the model ends at 7.80.

* This is the final output for this engine.

| Color       | Meaning                          |
| ----------- | -------------------------------- |
| **Red**  | Feature increases the prediction |
| **Blue** | Feature decreases the prediction |

**sensor_9_roll_std_12 = 4.44**

* This is the strongest contributor increasing the prediction

* It measures rolling standard deviation of sensor 9 over a window of 12

* High rolling standard deviation means:

   Sensor values are highly unstable

  Indicates abnormal operating conditions

  Because of this instability, the model pushes the prediction upward (towards failure risk)

📌 Interpretation:

High fluctuation in sensor 9 strongly signals possible failure.

**sensor_14_roll_std_12 = 3.05**

* Also increases the prediction

* Indicates variability in sensor 14 readings

* Less impact than sensor 9, but still significant

📌 Interpretation:

Moderate instability in sensor 14 adds to the failure risk.

**Blue features (right side)**

* These features pull the prediction down

* They represent:

    Stable sensors

    Normal operating conditions

    However, their combined effect is weaker than the red features


In [ ]:
#Save SHAP Values
dump(shap_values, "shap_values.pkl")
dump(X_sample, "shap_sample_data.pkl")

## **Task 4: Deployment**

In [ ]:
#Save the Trained Model
import joblib

joblib.dump(
    best_xgb_grid,
    "/content/predictive_maintenance_model.pkl"
)

print("Saved!")
print(os.listdir("/content"))

In [ ]:
import json

sample_input = X_train.iloc[0].to_dict()

with open("sample_input.json", "w") as f:
    json.dump(sample_input, f)

In [ ]:
from google.colab import files
files.download("sample_input.json")

##### **Build Flask API**

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
from flask import Flask, request, jsonify
import time
import joblib

In [ ]:
ngrok.set_auth_token("37s4tpKvpHYzZDTZaR5fojyXlc4_48vCbozcVRERUDWzfLHtW")

In [ ]:
# Load model
model = joblib.load("/content/predictive_maintenance_model.pkl")
print("Model loaded successfully")

In [ ]:
#Save feature list ONCE
feature_columns = list(X_train.columns)
joblib.dump(feature_columns, "feature_columns.pkl")

In [ ]:
#Load feature list in Flask
feature_columns = joblib.load("/content/feature_columns.pkl")

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import time
import joblib
import pandas as pd

# ========== INIT ==========
app = Flask(__name__)

# Load model & feature list
model = joblib.load("/content/predictive_maintenance_model.pkl")

print("Model & feature columns loaded")

# ========== ROUTE ==========
@app.route("/predict", methods=["POST"]) #Creates an API URL & It accepts only POST requests(This means you send data to the API (not via browser link))
def predict(): #This function runs whenever someone sends data to /predict
    try:
        start_time = time.time() #Records the current time

        data = request.get_json() #Reads the incoming JSON data
        if data is None:
            return jsonify({"error": "No JSON received"}), 400

        input_df = pd.DataFrame([data]) #Converts JSON into a pandas DataFrame

        # Add missing features with default 0
        for col in feature_columns:
            if col not in input_df.columns:
                input_df[col] = 0

        # Keep only training features in correct order
        input_df = input_df[feature_columns]

        failure_prob = model.predict_proba(input_df)[0][1]

        latency = (time.time() - start_time) * 1000 #Finds how long the API took to respond & Converts seconds → milliseconds

        return jsonify({
            "failure_probability": round(float(failure_prob), 4),
            "latency_ms": round(latency, 2)
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ========== RUN ==========
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    app.run(host="0.0.0.0", port=5000, debug=False)